# Answering Business Questions using SQL

In [1]:
%load_ext sql
%sql sqlite:///chinook.db

'Connected: @chinook.db'

In [2]:
%%sql
SELECT
    name,
    type
FROM sqlite_master
WHERE type IN ("table","view");

 * sqlite:///chinook.db
Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


In [3]:
%%sql
SELECT * from album
LIMIT 5

 * sqlite:///chinook.db
Done.


album_id,title,artist_id
1,For Those About To Rock We Salute You,1
2,Balls to the Wall,2
3,Restless and Wild,2
4,Let There Be Rock,1
5,Big Ones,3


In [4]:
%%sql
SELECT * from customer
LIMIT 5

 * sqlite:///chinook.db
Done.


customer_id,first_name,last_name,company,address,city,state,country,postal_code,phone,fax,email,support_rep_id
1,Luís,Gonçalves,Embraer - Empresa Brasileira de Aeronáutica S.A.,"Av. Brigadeiro Faria Lima, 2170",São José dos Campos,SP,Brazil,12227-000,+55 (12) 3923-5555,+55 (12) 3923-5566,luisg@embraer.com.br,3
2,Leonie,Köhler,None,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,+49 0711 2842222,None,leonekohler@surfeu.de,5
3,François,Tremblay,None,1498 rue Bélanger,Montréal,QC,Canada,H2G 1A7,+1 (514) 721-4711,None,ftremblay@gmail.com,3
4,Bjørn,Hansen,None,Ullevålsveien 14,Oslo,None,Norway,0171,+47 22 44 22 22,None,bjorn.hansen@yahoo.no,4
5,František,Wichterlová,JetBrains s.r.o.,Klanova 9/506,Prague,None,Czech Republic,14700,+420 2 4172 5555,+420 2 4172 5555,frantisekw@jetbrains.com,4


In [5]:
%%sql
SELECT * from employee
LIMIT 5

 * sqlite:///chinook.db
Done.


employee_id,last_name,first_name,title,reports_to,birthdate,hire_date,address,city,state,country,postal_code,phone,fax,email
1,Adams,Andrew,General Manager,None,1962-02-18 00:00:00,2016-08-14 00:00:00,11120 Jasper Ave NW,Edmonton,AB,Canada,T5K 2N1,+1 (780) 428-9482,+1 (780) 428-3457,andrew@chinookcorp.com
2,Edwards,Nancy,Sales Manager,1,1958-12-08 00:00:00,2016-05-01 00:00:00,825 8 Ave SW,Calgary,AB,Canada,T2P 2T3,+1 (403) 262-3443,+1 (403) 262-3322,nancy@chinookcorp.com
3,Peacock,Jane,Sales Support Agent,2,1973-08-29 00:00:00,2017-04-01 00:00:00,1111 6 Ave SW,Calgary,AB,Canada,T2P 5M5,+1 (403) 262-3443,+1 (403) 262-6712,jane@chinookcorp.com
4,Park,Margaret,Sales Support Agent,2,1947-09-19 00:00:00,2017-05-03 00:00:00,683 10 Street SW,Calgary,AB,Canada,T2P 5G3,+1 (403) 263-4423,+1 (403) 263-4289,margaret@chinookcorp.com
5,Johnson,Steve,Sales Support Agent,2,1965-03-03 00:00:00,2017-10-17 00:00:00,7727B 41 Ave,Calgary,AB,Canada,T3B 1Y7,1 (780) 836-9987,1 (780) 836-9543,steve@chinookcorp.com


# Checking which genre are sold most in USA

In [6]:
%%sql
SELECT * from genre
LIMIT 5

 * sqlite:///chinook.db
Done.


genre_id,name
1,Rock
2,Jazz
3,Metal
4,Alternative & Punk
5,Rock And Roll


In [7]:
%%sql
SELECT * from track
LIMIT 5

 * sqlite:///chinook.db
Done.


track_id,name,album_id,media_type_id,genre_id,composer,milliseconds,bytes,unit_price
1,For Those About To Rock (We Salute You),1,1,1,"Angus Young, Malcolm Young, Brian Johnson",343719,11170334,0.99
2,Balls to the Wall,2,2,1,None,342562,5510424,0.99
3,Fast As a Shark,3,2,1,"F. Baltes, S. Kaufman, U. Dirkscneider & W. Hoffman",230619,3990994,0.99
4,Restless and Wild,3,2,1,"F. Baltes, R.A. Smith-Diesel, S. Kaufman, U. Dirkscneider & W. Hoffman",252051,4331779,0.99
5,Princess of the Dawn,3,2,1,Deaffy & R.A. Smith-Diesel,375418,6290521,0.99


In [8]:
%%sql
SELECT * from invoice_line
LIMIT 5

 * sqlite:///chinook.db
Done.


invoice_line_id,invoice_id,track_id,unit_price,quantity
1,1,1158,0.99,1
2,1,1159,0.99,1
3,1,1160,0.99,1
4,1,1161,0.99,1
5,1,1162,0.99,1


## Isolating USA sales

In [9]:
%%sql
WITH usa_sales AS
    (
        SELECT il.* FROM invoice_line il
        INNER JOIN invoice i ON i.invoice_id = il.invoice_id
        INNER JOIN customer c ON c.customer_id = i.customer_id
        WHERE c.country = "USA"
    )

SELECT
    g.name genre_name,
    SUM(us.quantity) number_of_tracks
FROM genre g
INNER JOIN track t ON t.genre_id = g.genre_id
INNER JOIN usa_sales us ON us.track_id = t.track_id
GROUP BY 1
ORDER BY number_of_tracks DESC
LIMIT 10

 * sqlite:///chinook.db
Done.


genre_name,number_of_tracks
Rock,561
Alternative & Punk,130
Metal,124
R&B/Soul,53
Blues,36
Alternative,35
Latin,22
Pop,22
Hip Hop/Rap,20
Jazz,14


### Based on the observations from the above table Punk, Blues and Pop are more sold in USA compared to Hip-hop

### So the best singers will be Red Tone, Meteor and the Girls and Slim Jim Bites	

# Analyzing the purchases of customers belonging to each employee

In [10]:
%%sql
SELECT
    e.employee_id emp_id,
    e.first_name||" "||e.last_name emp_name,
    SUM(i.total) dollar_amount
FROM employee e
INNER JOIN customer c ON c.support_rep_id = e.employee_id
INNER JOIN invoice i ON i.customer_id = c.customer_id
GROUP BY 1
ORDER BY 3 DESC
LIMIT 10

 * sqlite:///chinook.db
Done.


emp_id,emp_name,dollar_amount
3,Jane Peacock,1731.5100000000039
4,Margaret Park,1584.0000000000034
5,Steve Johnson,1393.920000000002


## Let's differentiate them basing on their hired dates

In [11]:
%%sql
SELECT
    e.first_name||" "||e.last_name emp_name,
    e.hire_date hire_date,
    SUM(i.total) dollar_amount
FROM employee e
INNER JOIN customer c ON c.support_rep_id = e.employee_id
INNER JOIN invoice i ON i.customer_id = c.customer_id
GROUP BY e.employee_id
ORDER BY 3 DESC
LIMIT 10

 * sqlite:///chinook.db
Done.


emp_name,hire_date,dollar_amount
Jane Peacock,2017-04-01 00:00:00,1731.5100000000039
Margaret Park,2017-05-03 00:00:00,1584.0000000000034
Steve Johnson,2017-10-17 00:00:00,1393.920000000002


### As we can see there are differences in sold amount but it can be justified using the hire_date column
### Those who were a late hire also sold less

# Analyzing the sales data for customers from each different country

In [12]:
%%sql
WITH order_other AS
    (
        SELECT
        c.customer_id customer_id,
        CASE
            WHEN (
                SELECT COUNT(*) FROM customer
                WHERE country = c.country
            ) = 1 THEN "Other"
            ELSE c.country
        END AS country,
        il.*
        FROM customer c
        INNER JOIN invoice i ON i.customer_id = c.customer_id
        INNER JOIN invoice_line il ON il.invoice_id = i.invoice_id
    )
    
SELECT
    country,
    number_of_customers,
    total_sales,
    average_sales,
    average_order
FROM
    (
        SELECT
            country,
            COUNT(distinct customer_id) number_of_customers,
            SUM(unit_price) total_sales,
            SUM(unit_price)/COUNT(distinct customer_id) average_sales,
            SUM(unit_price)/COUNT(distinct invoice_id) average_order,
            CASE
                WHEN country = "Other" THEN 1
                ELSE 0
            END AS sort
        FROM order_other
        GROUP BY country
        ORDER BY sort ASC, number_of_customers DESC
    )

 * sqlite:///chinook.db
Done.


country,number_of_customers,total_sales,average_sales,average_order
USA,13,1040.490000000008,80.03769230769292,7.942671755725252
Canada,8,535.5900000000034,66.94875000000043,7.047236842105309
Brazil,5,427.68000000000245,85.53600000000048,7.011147540983647
France,5,389.0700000000021,77.81400000000042,7.781400000000042
Germany,4,334.6200000000016,83.6550000000004,8.161463414634186
United Kingdom,3,245.52000000000078,81.84000000000026,8.768571428571457
Czech Republic,2,273.24000000000103,136.62000000000052,9.108000000000034
India,2,183.1500000000002,91.5750000000001,8.72142857142858
Portugal,2,185.13000000000022,92.56500000000011,6.3837931034482835
Other,15,1094.9400000000085,72.99600000000056,7.448571428571486


### Based on the above data it can be inferred that the market is low in Czech Republic, Portugal and India. So they may be a good market choice.

# Differentiating an album purchase to a track puchase to help the company make an administrative decision

In [17]:
%%sql

WITH invoice_first_track AS
    (
        SELECT
            invoice_id,
            MIN(track_id) first_track_id
        FROM invoice_line il
        GROUP BY 1
    )

SELECT
    album_purchase,
    COUNT(invoice_id) number_of_invoices,
    CAST(COUNT(invoice_id) AS Float)/(SELECT COUNT(*) FROM invoice) percentage
FROM
    (
        SELECT
            CASE
                    WHEN 
                        (
                            SELECT t.track_id FROM track t
                            WHERE t.album_id = (SELECT t1.album_id from track t1
                                              WHERE t1.track_id = ifs.first_track_id)
                            EXCEPT
                            SELECT track_id FROM invoice_line
                            WHERE invoice_id = ifs.invoice_id
                        ) IS NULL
                    AND
                        (
                            SELECT track_id FROM invoice_line
                            WHERE invoice_id = ifs.invoice_id
                            EXCEPT
                            SELECT track_id FROM track t
                            WHERE album_id = (SELECT album_id from track
                                              WHERE track_id = ifs.first_track_id)
                        ) IS NULL
                    THEN "yes"
                    ELSE "no"
            END AS album_purchase,
            ifs.*
        FROM invoice_first_track ifs
    )
GROUP BY 1

 * sqlite:///chinook.db
Done.


album_purchase,number_of_invoices,percentage
no,500,0.8143322475570033
yes,114,0.18566775244299674


### FRom the above analysis it can be said that stopping the purchase of whole albums manually may result in the lostt of 19% in revenue